# 2. Data Preparation
📒 `2.0-rc-data-preparation.ipynb`

**Objetivo:** Transformar os dados brutos em um formato para uso em análise exploratória e modelagem.

⚙️ **Atividades:**
- Limpeza dos dados
- Separação dos Dados de Treinamento e Teste
- Tratamento de Variáveis Categóricas
- Normalização/Escalonamento das Variáveis Numéricas
- Tratamento de Desbalanceamento das Classes
- Feature engineering
- Seleção de Features
- Preparação de Pipelines (ColumnTransformer)
- Divisão para Validação (Cross Validation k-fold)
- Salvamento do dataset (`data/processed/`)


In [238]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src._utils import detect_outliers_iqr, plot_outliers, handle_outliers, analyze_outliers

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Configura o matplotlib para mostrar gráficos inline
%matplotlib inline

# Setup para mostrar todas as colunas do dataframe
pd.set_option('display.max_columns', None)

# Carrega dados
df = pd.read_csv("../data/raw/bootcamp_train.csv")
print( "N° de linhas e colunas: ", df.shape)
df.head()

N° de linhas e colunas:  (35260, 15)


,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta,falha_maquina,FDF (Falha Desgaste Ferramenta),FDC (Falha Dissipacao Calor),FP (Falha Potencia),FTE (Falha Tensao Excessiva),FA (Falha Aleatoria)
0,0,L56434,L,298.3,309.1,90.0,1616.0,31.1,195.0,não,False,False,Não,False,Não
1,1,L48741,L,298.2,308.4,90.0,1388.0,53.8,137.0,Não,False,False,Não,False,Não
2,2,L48850,L,298.2,307.8,90.0,1528.0,31.1,NaN,Não,N,False,Não,False,Não
3,3,M20947,M,300.9,310.8,90.0,1599.0,33.0,7.0,não,False,False,Não,False,não
4,4,L53849,L,-36.0,310.5,90.0,1571.0,33.9,NaN,não,N,False,não,False,Não


## 1. Limpeza dos Dados


### a) Remove colunas desnecessárias e renomeia as colunas

In [239]:
# Carrega dados
df = pd.read_csv("../data/raw/bootcamp_train.csv")

# Deleta as colunas que não são necessárias
df.drop(columns=['id', 'id_produto'], inplace=True)

# Renomeia as colunas
cols_to_rename = {
    'tipo': 'tipo',
    'temperatura_ar': 'temperatura_ar_[K]',
    'temperatura_processo': 'temperatura_processo_[K]',
    'umidade_relativa': 'umidade_relativa_[%]',
    'velocidade_rotacional': 'velocidade_rotacional_[rpm]',
    'torque': 'torque_[Nm]',
    'desgaste_da_ferramenta': 'desgaste_da_ferramenta_[mm]',
    'falha_maquina': 'falha_maquina',
    'FTE (Falha Tensao Excessiva)': 'FTE',
    'FDC (Falha Dissipacao Calor)': 'FDC',
    'FP (Falha Potencia)': 'FP',
    'FDF (Falha Desgaste Ferramenta)': 'FDF',
    'FA (Falha Aleatoria)': 'FA'
}

# Renomeia as colunas
df.rename(columns=cols_to_rename, inplace=True)


# Visualiza as 5 primeiras linhas
print( "N° de linhas e colunas: ", df.shape)
print("Visualiza as 5 primeiras linhas:")
df.head()


N° de linhas e colunas:  (35260, 13)
Visualiza as 5 primeiras linhas:


,tipo,temperatura_ar_[K],temperatura_processo_[K],umidade_relativa_[%],velocidade_rotacional_[rpm],torque_[Nm],desgaste_da_ferramenta_[mm],falha_maquina,FDF,FDC,FP,FTE,FA
0,L,298.3,309.1,90.0,1616.0,31.1,195.0,não,False,False,Não,False,Não
1,L,298.2,308.4,90.0,1388.0,53.8,137.0,Não,False,False,Não,False,Não
2,L,298.2,307.8,90.0,1528.0,31.1,NaN,Não,N,False,Não,False,Não
3,M,300.9,310.8,90.0,1599.0,33.0,7.0,não,False,False,Não,False,não
4,L,-36.0,310.5,90.0,1571.0,33.9,NaN,não,N,False,não,False,Não



### b) Valores Nulos

In [240]:
# Verifica dados Nulos
df.isnull().sum()

tipo                             0
temperatura_ar_[K]             616
temperatura_processo_[K]       599
umidade_relativa_[%]             0
velocidade_rotacional_[rpm]    751
torque_[Nm]                    623
desgaste_da_ferramenta_[mm]    952
falha_maquina                    0
FDF                              0
FDC                              0
FP                               0
FTE                              0
FA                               0
dtype: int64

In [241]:
# Tratando Valores Nulos
# Preenchimento de valores ausentes com a mediana ( menos sensivel aos outliers)

num_cols_to_inspect = [
    'temperatura_ar_[K]', 
    'temperatura_processo_[K]', 
    'umidade_relativa_[%]', 
    'velocidade_rotacional_[rpm]', 
    'torque_[Nm]', 
    'desgaste_da_ferramenta_[mm]'
    ]

for col in num_cols_to_inspect:
    df[col].fillna(df[col].median(), inplace=True)

print("Valores Nulos tratados com a mediana: \n", df.isnull().sum())


Valores Nulos tratados com a mediana: 
 tipo                           0
temperatura_ar_[K]             0
temperatura_processo_[K]       0
umidade_relativa_[%]           0
velocidade_rotacional_[rpm]    0
torque_[Nm]                    0
desgaste_da_ferramenta_[mm]    0
falha_maquina                  0
FDF                            0
FDC                            0
FP                             0
FTE                            0
FA                             0
dtype: int64


### c) Inconsistência nos dados

In [242]:
# Verifica dados Inconsistentes
print("Descrição dos dados numéricos: \n", df.describe())
print("-"*50)

for col in num_cols_to_inspect:
    print(df.loc[df[col] < 0, col].value_counts())
    print("\n")

Descrição dos dados numéricos: 
        temperatura_ar_[K]  temperatura_processo_[K]  umidade_relativa_[%]  \
count        35260.000000              35260.000000          35260.000000   
mean           270.060479                280.956146             89.997672   
std             95.578134                 96.191402              0.142191   
min            -36.000000                -38.000000             80.590429   
25%            298.000000                308.500000             90.000000   
50%            299.600000                309.800000             90.000000   
75%            301.000000                310.900000             90.000000   
max            304.500000                313.800000             94.575256   

       velocidade_rotacional_[rpm]   torque_[Nm]  desgaste_da_ferramenta_[mm]  
count                 35260.000000  35260.000000                 35260.000000  
mean                   1382.383834     40.211941                    74.903176  
std                     489.03371

In [243]:
# Trata Inconsistências
for col in num_cols_to_inspect:
    df.loc[df[col] < 0, col] = df[col].median()

print("Inconsistências tratadas!")

Inconsistências tratadas!


In [244]:
cols_to_inspect = [
    'falha_maquina', 
    'FDF', 
    'FDC', 
    'FP', 
    'FTE', 
    'FA'
    ]

# Verifica valores únicos em todas as colunas
for col in cols_to_inspect:
    print(f"Coluna: {col}")
    print(f"Valores únicos: {df[col].unique()}")
    print(f"Quantidade de valores únicos: {df[col].nunique()}")
    print("-"*50)


Coluna: falha_maquina
Valores únicos: ['não' 'Não' 'sim' 'N' '0' 'Sim' 'y' '1']
Quantidade de valores únicos: 8
--------------------------------------------------
Coluna: FDF
Valores únicos: ['False' 'N' '0' 'True' '-' '1']
Quantidade de valores únicos: 6
--------------------------------------------------
Coluna: FDC
Valores únicos: ['False' 'nao' '0' 'True' 'y' '1']
Quantidade de valores únicos: 6
--------------------------------------------------
Coluna: FP
Valores únicos: ['Não' 'não' 'Sim' 'N' '0' 'sim' '1' 'y']
Quantidade de valores únicos: 8
--------------------------------------------------
Coluna: FTE
Valores únicos: [False  True]
Quantidade de valores únicos: 2
--------------------------------------------------
Coluna: FA
Valores únicos: ['Não' 'não' '0' '-' 'Sim' 'sim' '1']
Quantidade de valores únicos: 7
--------------------------------------------------


In [245]:
# Dicionário de mapeamento para padronizar valores binários
map_classes = {
    # Valores que representam "0" (não/falso)
    'não': '0',
    'Não': '0',
    'N': '0',
    'False': '0',
    'false': '0',
    'nao': '0',
    '-': '0',
    # Valores que representam "1" (sim/verdadeiro)
    'sim': '1',
    'Sim': '1',
    'y': '1',
    'True': '1',
    'true': '1'
}

# Lista de colunas para aplicar a transformação
cols_to_transform = [
    'falha_maquina',
    'FDF',
    'FDC',
    'FP',
    'FTE',
    'FA'
]

# Aplica a transformação em cada coluna
for col in cols_to_transform:
    df[col] = df[col].astype(str)
    # Aplica o mapeamento
    df[col] = df[col].map(lambda x: map_classes.get(x, x))
    # Converte a coluna para tipo numérico
    df[col] = pd.to_numeric(df[col])

# Verifica os valores únicos após a transformação
print("Verificação após a transformação:")
print("-" * 50)
for col in cols_to_transform:
    print(f"Coluna: {col}")
    print(f"Valores únicos: {df[col].unique()}")
    print(f"Quantidade: {df[col].nunique()}")
    print("-" * 50)



Verificação após a transformação:
--------------------------------------------------
Coluna: falha_maquina
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------
Coluna: FDF
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------
Coluna: FDC
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------
Coluna: FP
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------
Coluna: FTE
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------
Coluna: FA
Valores únicos: [0 1]
Quantidade: 2
--------------------------------------------------


### d) Dados Duplicados

In [ ]:
print("Dados Duplicados:", df.duplicated().sum())

### e) Análise de Outliers

In [ ]:
numeric_cols = [
    'temperatura_ar_[K]',
    'temperatura_processo_[K]',
    'umidade_relativa_[%]',
    'velocidade_rotacional_[rpm]',
    'torque_[Nm]',
    'desgaste_da_ferramenta_[mm]'
]

# máscara acumulada de "é outlier em QUALQUER coluna"
any_outlier = pd.Series(False, index=df.index)

for col in numeric_cols:
    mask, stats = detect_outliers_iqr(df[col], col)
    plot_outliers(df[col], col, mask, stats)
    any_outlier |= mask  # acumula outliers

# remover linhas que são outlier em qualquer coluna
df_clean = df.loc[~any_outlier].copy()

print(f"Linhas originais: {len(df)}")
print(f"Linhas removidas (outliers): {any_outlier.sum()}")
print(f"Linhas após limpeza: {len(df_clean)}")

# Análise resumida
outlier_analysis = analyze_outliers(df, numeric_cols) 
print(outlier_analysis.to_string(index=False))


In [ ]:
df_clean.info()

### Visualiza os Dados com Pandas Profile Report

In [ ]:
# Visualiza os dados pós tratamento com Pandas Profiling
from ydata_profiling import ProfileReport

FILE_PATH = "../reports/profile-reports/data_cleaned.html"

profile = ProfileReport(
    df_clean,
    title="Profiling Report - Dados Tratados"
)

# Salva HTML estático
profile.to_file(FILE_PATH)

In [ ]:
# Salva o dataset tratado
path_to_save = "../data/processed/data_cleaned.csv"
df_clean.to_csv(path_to_save, index=False)

## 2. Pré Processamento

In [ ]:
# === Imports ===
import os
import numpy as np
import pandas as pd
import json

 #sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate # Divisão de dados
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler # Transformações de pré-processamento
from sklearn.compose import ColumnTransformer # Combina transformações de pré-processamento
from sklearn.pipeline import Pipeline # Pipeline de pré-processamento

from joblib import dump

# Configuração
RANDOM_SEED = 42



In [ ]:
# Carrega o dataset tratado
df = pd.read_csv('../data/processed/data_cleaned.csv')

# Remove as colunas id e id_produto
df.drop(columns=['id','id_produto'],inplace=True)

df.head()

In [ ]:
# Variáveis Alvo (Y)
target_vars = ['falha_maquina', 
               'FTE', 
               'FDC', 
               'FP', 
               'FDF', 
               'FA']

# Seleciona os atributos numéricos e categóricos (X) e a variável alvo (y)
X = df.drop(columns=target_vars).copy()
y = df['falha_maquina'].copy()

# # Identifica e seleciona os atributos numéricos e categóricos
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categoric_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

# Divide os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y)

# Pré-processamento em Pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categoric_cols) # Ignora valores desconhecidos
],
    remainder='drop' # Deleta as colunas que não foram transformadas
)

print(f"Variável Alvo (y): {y.name}")
print(f"\nAtributos Numéricos (X): {numeric_cols}")
print(f"\nAtributos Categóricas (X): {categoric_cols}")


### Salva os artefatos e dados tratados 

In [ ]:

# Persistência dos dados tratados
os.makedirs('../models/artifacts', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

# Salvar os splits para manter os dados tratados
X_train.to_csv('../data/interim/X_train.csv', index=False)
X_test.to_csv('../data/interim/X_test.csv', index=False)
y_train.to_csv('../data/interim/y_train.csv', index=False)
y_test.to_csv('../data/interim/y_test.csv', index=False)


# Salvar “especificação” do pré-processamento (listas de colunas + objeto definidos/ sem ajuste)
dump({
    "numeric_cols": numeric_cols,
    "categorical_cols": categoric_cols,
    "preprocessor": preprocessor,
    "random_seed": RANDOM_SEED,
    "target_name": "falha_maquina",
    "target_vars_all": target_vars
}, "../models/artifacts/preprocessing_spec.joblib")